In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

The goal of this topic is to determine the ideal projection when the dimensions of the input may vary (in columns). The dimensions of the columns are determined by feature generation/embeddings. 

Let:

*  $W$ be the projection matrix
*  $X$ to be the matrix representing input data which may vary in dimension from iteration to iteration
*  $Y$ be the result of the projection, i.e. $WX = Y$

WLOG fix the size of $Y$, (let's call this $d$). Then we have $W \in \mathbb{R}^{d \times n}$.

In this setting within the RJMCMC framework, the number of parameters we are estimating is:

*  all entries in $W$ ($d \times n$)  
*  all relevant combination of feature generation functions and their respective parameters 
                                             

### Examples of feature vector generation functions

As an example of an algorithm which we should compare is the multivariate adaptive regression splines (MARS)

**Hinge Function**

The hinge function ($f_\text{hinge}$) is similiar to ReLu), can be defined as a feature generation function, (_probably not the right way to write this out_)

$$ f_{\text{hinge}^+} (X_{j}, \theta) = (X_j-\theta \mathbf{1})_{+}$$

Where $X_j$ represents the $j$th column, without loss of generality we can likewise define $f_{\text{hinge}^-} (X_{j}, \theta) = (\theta \mathbf{1} - X_j)_{+}$

**Interaction**

Interaction term will be defined the dot product of two feature vectors (which can be the same feature vector), (_probably not the right way to write this out_)

$$ f_\text{interaction} (X_1, X_2) = X_1  \cdot X_2 $$

Replicating MARS in RJMCMC
--------------------------

[MARS](https://en.wikipedia.org/wiki/Multivariate_adaptive_regression_splines) is a prime candidate for exploring this problem. 

In the RJMCMC space we have the following decisions:

1. GROW - we can grow the state by Proposing a new feature vector generation transformation and respective parameter, e.g. hinge, interaction, or more complex function transforms which could be a composition of function generation...
2. DESTROY - we can also delete a created feature

This would be represent moving from one state to another.                                  

In [3]:
import pandas as pd
import numpy as np

from truncmvn import multivariate_trunc_normal   

In [4]:
ndim = 10
mean = (np.random.rand(ndim)-1)*3
bounds=np.ones((ndim, 2))
bounds[:, 0] = -3
bounds[:, 1] = 3
theta = multivariate_trunc_normal(bounds, mean)

c:\users\chapm\anaconda3\lib\site-packages\emcee\ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
c:\users\chapm\anaconda3\lib\site-packages\emcee\ensemble.py:336: RuntimeWarning: invalid value encountered in greater
  accept = (lnpdiff > np.log(self._random.rand(len(lnpdiff))))


Before starting, we will prepare and test the data...
We will try to make the implementation in line with expectations around scikit-learn

In [13]:
trees = pd.read_csv("trees.csv")
trees.head()

,Girth,Height,Volume
0,8.3,70.0,10.3
1,8.6,65.0,10.3
2,8.8,63.0,10.2
3,10.5,72.0,16.4
4,10.7,81.0,18.8


In [14]:
# we have a maximum size that our design - set it WLOG to be 3 times dim of current dataset
max_search = trees.shape[1] * 3

In [21]:
# assume that everything is independent 
# and build statistics...
cov = np.diag(trees[['Girth', 'Height']].var(0))

